In [2]:
from EQTransformer.core.trainer import trainer
trainer(input_hdf5='../data/STEAD/merge_modified.hdf5',
        input_csv='../data/STEAD/mix_train_merge_modified.csv',
        output_name='test_trainer',                
        cnn_blocks=4,
        lstm_blocks=2,
        padding='same',
        activation='relu',
        label_type='gaussian',
        mode='generator',
        train_valid_test_split=[0.80, 0.10, 0.10],
        batch_size=20,
        epochs=10, 
        patience=2,
        gpuid=None,
        gpu_limit=None)

Learning rate:  0.001
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 6000, 3)      0                                            
__________________________________________________________________________________________________
separable_conv1d_21 (SeparableC (None, 6000, 8)      65          input[0][0]                      
__________________________________________________________________________________________________
max_pooling1d_6 (MaxPooling1D)  (None, 2000, 8)      0           separable_conv1d_21[0][0]        
__________________________________________________________________________________________________
separable_conv1d_22 (SeparableC (None, 2000, 16)     216         max_pooling1d_6[0][0]            
______________________________________________________________________

In [4]:
from EQTransformer.core.tester import tester
tester(input_hdf5='../data/STEAD/merge_modified.hdf5',
       input_testset='test_trainer_outputs/test.npy',
       input_model='test_trainer_outputs/final_model.h5',
       output_name='test_tester',
       detection_threshold=0.50,                
       P_threshold=0.3,
       S_threshold=0.3, 
       number_of_plots=3,
       estimate_uncertainty=False, 
       number_of_sampling=2,
       input_dimention=(6000, 3),
       normalization_mode='std',
       mode='generator',
       batch_size=10,
       gpuid=None,
       gpu_limit=None)

Loading the model ...!
Loading is complete!!!
Testing ...
Writting results into: " test_tester_outputs "


100%|██████████| 1000/1000 [18:18<00:00,  1.18s/it]

In [18]:
import tensorflow as tf
tf.gfile = tf.compat.v1.gfile
tf.GraphDef = tf.compat.v1.GraphDef
tf.profiler = tf.compat.v1.profiler
run_meta = tf.compat.v1.RunMetadata()

def load_pb(pb_model):
    with tf.gfile.GFile(pb_model, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='')
        return graph

def estimate_flops(pb_model):
    graph = load_pb(pb_model)
    with graph.as_default():
        # placeholder input would result in incomplete shape. So replace it with constant during model frozen.
        flops = tf.profiler.profile(graph, run_meta=run_meta, cmd='op', options=tf.profiler.ProfileOptionBuilder.float_operation())
        print('Model {} needs {} FLOPS after freezing'.format(pb_model, flops.total_float_ops))

model = "test_trainer_outputs/final_model.h5"
estimate_flops(model)

Model test_trainer_outputs/final_model.h5 needs 0 FLOPS after freezing


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: Unexpected end-group tag: Not all data was converted
  # Remove the CWD from sys.path while we load stuff.


In [31]:
import numpy as np

parameters = np.array([373495, 184416, 229335, 64320, 39776])
FLOPs = np.array([79687040, (1478880+3465216)*4, (15590240+204544)*4, (1478880+204544)*4, 5271488])

Detections_F1 = np.array([0.9973, 0.9962, 0.9995, 0.9974, 0.9949])
P_F1 = np.array([0.9942, 0.9922, 0.9966, 0.9926, 0.9896])
S_F1 = np.array([0.9731, 0.9731, 0.9936, 0.9821, 0.9710])
F1 = (Detections_F1 + P_F1 + S_F1)/3

ID_D = Detections_F1/parameters
ID_P = P_F1/parameters
ID_S = S_F1/parameters
ID = (ID_D + ID_P + ID_S)/3
FLOPs

array([79687040, 19776384, 63179136,  6733696,  5271488])

In [37]:
(0.90+0.61+0.60)/3/108691

6.470943623053733e-06

In [54]:
def Netscore(a, p, m):
    netscore = list()
    for idx in range(len(a)):
        netscore.append(20*np.log10((a[idx]*100)**2/((p[idx]**0.5)*(m[idx]**0.1))))
    return netscore

Netscore(F1, parameters, FLOPs)

[8.268170669254495,
 12.525339538820553,
 10.734403506796594,
 18.097719150698445,
 20.30031084453737]

In [59]:
Netscore([(0.90+0.61+0.60)/3], [108691], [7806976])

[9.739546490052755]

In [72]:
def Ideal_Netscore(a, p, m):
    netscore = list()
    for idx in range(len(a)):
        netscore.append(20*np.log10((a[idx]*100)**2/((p[idx]**0.5)*(m[idx]**0.1))))
    return netscore

Ideal_Netscore([1], [100000000], [1])

[0.0]